# Desafio 6

Neste desafio, vamos praticar _feature engineering_, um dos processos mais importantes e trabalhosos de ML. Utilizaremos o _data set_ [Countries of the world](https://www.kaggle.com/fernandol/countries-of-the-world), que contém dados sobre os 227 países do mundo com informações sobre tamanho da população, área, imigração e setores de produção.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
from sklearn.preprocessing  import KBinsDiscretizer, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
# Algumas configurações para o matplotlib.
#%matplotlib inline

from IPython.core.pylabtools import figsize


figsize(12, 8)

sns.set()

In [3]:
countries = pd.read_csv("countries.csv", decimal=',')

In [4]:
new_column_names = [
    "Country", "Region", "Population", "Area", "Pop_density", "Coastline_ratio",
    "Net_migration", "Infant_mortality", "GDP", "Literacy", "Phones_per_1000",
    "Arable", "Crops", "Other", "Climate", "Birthrate", "Deathrate", "Agriculture",
    "Industry", "Service"
]

#pré-processamento
countries.columns = new_column_names
try:
    for i in range(0, countries['Region'].shape[0]):
                   countries['Region'][i] = countries['Region'][i].strip()
except:
    pass 

countries.head(5)



C:\Users\darci\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Country,Region,Population,Area,Pop_density,Coastline_ratio,Net_migration,Infant_mortality,GDP,Literacy,Phones_per_1000,Arable,Crops,Other,Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,48.0,0.00,23.06,163.07,700.0,36.0,3.2,12.13,0.22,87.65,1.0,46.60,20.34,0.380,0.240,0.380
1,Albania,EASTERN EUROPE,3581655,28748,124.6,1.26,-4.93,21.52,4500.0,86.5,71.2,21.09,4.42,74.49,3.0,15.11,5.22,0.232,0.188,0.579
2,Algeria,NORTHERN AFRICA,32930091,2381740,13.8,0.04,-0.39,31.00,6000.0,70.0,78.1,3.22,0.25,96.53,1.0,17.14,4.61,0.101,0.600,0.298
3,American Samoa,OCEANIA,57794,199,290.4,58.29,-20.71,9.27,8000.0,97.0,259.5,10.00,15.00,75.00,2.0,22.46,3.27,NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,152.1,0.00,6.60,4.05,19000.0,100.0,497.2,2.22,0.00,97.78,3.0,8.71,6.25,NaN,NaN,NaN


In [5]:
countries.shape

(227, 20)

In [6]:
df_aux = pd.DataFrame({
                'tipos': countries.dtypes,
                'total': countries.shape[0],
                'faltantes':countries.isna().sum(),
                '% faltante': 100*countries.isna().sum() / countries.shape[0]
})

countries['Climate'].fillna(0, inplace=True)

df_aux

,tipos,total,faltantes,% faltante
Country,object,227,0,0.000000
Region,object,227,0,0.000000
Population,int64,227,0,0.000000
Area,int64,227,0,0.000000
Pop_density,float64,227,0,0.000000
Coastline_ratio,float64,227,0,0.000000
Net_migration,float64,227,3,1.321586
Infant_mortality,float64,227,3,1.321586
GDP,float64,227,1,0.440529
Literacy,float64,227,18,7.929515


## Observações

Esse _data set_ ainda precisa de alguns ajustes iniciais. Primeiro, note que as variáveis numéricas estão usando vírgula como separador decimal e estão codificadas como strings. Corrija isso antes de continuar: transforme essas variáveis em numéricas adequadamente.

Além disso, as variáveis `Country` e `Region` possuem espaços a mais no começo e no final da string. Você pode utilizar o método `str.strip()` para remover esses espaços.

## Inicia sua análise a partir daqui

## Questão 1

Quais são as regiões (variável `Region`) presentes no _data set_? Retorne uma lista com as regiões únicas do _data set_ com os espaços à frente e atrás da string removidos (mas mantenha pontuação: ponto, hífen etc) e ordenadas em ordem alfabética.

In [18]:
def q1():
    resp = countries['Region'].unique()
    return list(sorted(resp))
q1()

['ASIA (EX. NEAR EAST)',
 'BALTICS',
 'C.W. OF IND. STATES',
 'EASTERN EUROPE',
 'LATIN AMER. & CARIB',
 'NEAR EAST',
 'NORTHERN AFRICA',
 'NORTHERN AMERICA',
 'OCEANIA',
 'SUB-SAHARAN AFRICA',
 'WESTERN EUROPE']

## Questão 2

Discretizando a variável `Pop_density` em 10 intervalos com `KBinsDiscretizer`, seguindo o encode `ordinal` e estratégia `quantile`, quantos países se encontram acima do 90º percentil? Responda como um único escalar inteiro.

In [8]:
def q2():
    discr = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
    x = discr.fit_transform(countries['Pop_density'].values.reshape(-1,1))
    return int((x>=9).sum())
q2()

23

# Questão 3

Se codificarmos as variáveis `Region` e `Climate` usando _one-hot encoding_, quantos novos atributos seriam criados? Responda como um único escalar.

In [9]:
def q3():
    encoding = OneHotEncoder(sparse=False)
    encoded = encoding.fit_transform(countries[['Region', 'Climate']])

    return int(encoded.shape[1])
q3()

18

## Questão 4

Aplique o seguinte _pipeline_:

1. Preencha as variáveis do tipo `int64` e `float64` com suas respectivas medianas.
2. Padronize essas variáveis.

Após aplicado o _pipeline_ descrito acima aos dados (somente nas variáveis dos tipos especificados), aplique o mesmo _pipeline_ (ou `ColumnTransformer`) ao dado abaixo. Qual o valor da variável `Arable` após o _pipeline_? Responda como um único float arredondado para três casas decimais.

In [10]:
test_country = [
    'Test Country', 'NEAR EAST', -0.19032480757326514,
    -0.3232636124824411, -0.04421734470810142, -0.27528113360605316,
    0.13255850810281325, -0.8054845935643491, 1.0119784924248225,
    0.6189182532646624, 1.0074863283776458, 0.20239896852403538,
    -0.043678728558593366, -0.13929748680369286, 1.3163604645710438,
    -0.3699637766938669, -0.6149300604558857, -0.854369594993175,
    0.263445277972641, 0.5712416961268142
]


In [11]:
def q4():
    criando_pipe = Pipeline(steps=[
                                ('imputer',SimpleImputer(strategy="median")),
                                ('standardscaler', StandardScaler())
                                ])
    lista = []
    for i in countries.columns:
        if countries[i].dtypes == 'float64' or countries[i].dtypes == 'int64':
            lista.append(i)
        
    implementador = criando_pipe.fit(countries[lista])

    novo_df = pd.DataFrame([test_country], columns=countries.columns)
    resp = implementador.transform(novo_df[lista])[0][lista.index('Arable')].round(3)
    return float(resp)
q4()

-1.047

## Questão 5

Descubra o número de _outliers_ da variável `Net_migration` segundo o método do _boxplot_, ou seja, usando a lógica:

$$x \notin [Q1 - 1.5 \times \text{IQR}, Q3 + 1.5 \times \text{IQR}] \Rightarrow x \text{ é outlier}$$

que se encontram no grupo inferior e no grupo superior.

Você deveria remover da análise as observações consideradas _outliers_ segundo esse método? Responda como uma tupla de três elementos `(outliers_abaixo, outliers_acima, removeria?)` ((int, int, bool)).

In [12]:
def q5():
    countries['Net_migration'].dropna()
    q1,q3 = countries['Net_migration'].quantile([0.25, 0.75])
    iqr = q3 - q1
    out_baixo, out_alto = int((countries['Net_migration'] < q1 - 1.5*iqr).sum()), int((countries['Net_migration'] > q3 + 1.5*iqr).sum())
    aceitacao = bool (out_baixo/countries['Net_migration'].shape[0] < 0.05 or out_alto/countries['Net_migration'].shape[0] < 0.05)
    return out_baixo, out_alto, aceitacao
q5()

(24, 26, False)

In [13]:
#sns.boxplot(countries['Net_migration'], orient='vertical')
#sns.distplot(countries.Net_migration)

## Questão 6
Para as questões 6 e 7 utilize a biblioteca `fetch_20newsgroups` de datasets de test do `sklearn`

Considere carregar as seguintes categorias e o dataset `newsgroups`:

```
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
```


Aplique `CountVectorizer` ao _data set_ `newsgroups` e descubra o número de vezes que a palavra _phone_ aparece no corpus. Responda como um único escalar.

In [14]:
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)

In [15]:
def q6():
    vectorizer = CountVectorizer()
    x = vectorizer.fit_transform(newsgroup.data)
    dx = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names())
    return int (dx['phone'].sum())
q6()  

213

## Questão 7

Aplique `TfidfVectorizer` ao _data set_ `newsgroups` e descubra o TF-IDF da palavra _phone_. Responda como um único escalar arredondado para três casas decimais.

In [16]:
def q7():
    tf = TfidfVectorizer()
    news_tf = tf.fit_transform(newsgroup.data)
    dx = pd.DataFrame(news_tf.toarray(), columns=tf.get_feature_names())
    return float(dx['phone'].sum().round(3))
q7()

8.888